# Pandas Overview
When finished with this notebook, we'll be ready for anything pandas.


In [ ]:
import pandas as pd
from termcolor import colored # colored prints


## 1. Creating, Reading and Writing
Pandas has two core objects, **DataFrame** and **Series**.

### DateFrame
A DataFrame is a table. It contains an array of individual *entries*, each of which has a certain *value*. Each entry corresponds to a row (or *record*) and a *column*.

For example, consider the following simple DataFrame:

In [ ]:
pd.DataFrame({'Yes': [50, 21], 'No': [131, 2]})

Here's another example showing strings.

In [ ]:
pd.DataFrame({'Bob': ['I liked it.', 'It was awful.'], 'Sue': ['Pretty good.', 'Bland.']})

We are using the ```pd.DataFrame()``` constructor to generate these DataFrame objects. The syntax for declaring a new one is a dictionary whose keys are the column names (*Bob* and *Sue* in this example), and whose values are a list of entries. This is the standard way of constructing a new DataFrame, and the one you are most likely to encounter.


The dictionary-list constructor assigns values to the *column labels*, but just uses an ascending count from 0 (0, 1, 2, 3, ...) for the *row labels*. Sometimes this is OK, but oftentimes we will want to assign these labels ourselves.

The list of row labels used in a DataFrame is known as an **Index**. We can assign values to it by using an ```index``` parameter in our constructor:

In [ ]:
pd.DataFrame({'Bob': ['I liked it.', 'It was awful.'],
              'Sue': ['Pretty good.', 'Bland.']},
              index=['Product A', 'Product B'])

### Series
A Series, by contrast, is a sequence of data values. If a DataFrame is a table, a Series is a list. 

In [ ]:
pd.Series([1, 2, 3, 4, 5])

A Series is, in essence, a single column of a DataFrame. So you can assign column values to the Series the same way as before, using an ```index``` parameter. However, a Series does not have a column name, it only has one overall ```name```:

In [ ]:
pd.Series([30, 35, 40], index=['2015 Sales', '2016 Sales', '2017 Sales'], name='Product A')

The Series and the DataFrame are intimately related. It's helpful to think of a DataFrame as actually being just a bunch of Series "glued together". We'll see more of this below.

### Reading data files
Being able to create a DataFrame or Series by hand is handy. But, most of the time, we won't actually be creating our own data by hand. Instead, we'll be working with data that already exists.

Data can be stored in any of a number of different forms and formats. By far the most basic of these is the humble CSV file. When you open a CSV file you get something that looks like this:
```
Product A, Product B, Product C
30,21,9
35,34,1
41,11,11
```
Let's now set aside our toy datasets and read in a real dataset into a DataFrame. We'll use ```pd.read_csv()``` to do this.

In [ ]:
ign_scores = pd.read_csv("./datasets/data-vis/ign_scores.csv")

We can use the ```shape``` attribute to check how large a DataFrame is, and the ```head()``` function to peek the first five rows.

In [ ]:
print(ign_scores.shape)
ign_scores.head()

The ```pd.read_csv()``` function is well-endowed, with over 30 optional parameters you can specify, like being able to specify a specific index column using ```index_col.```

## 2. Indexing, Selecting, & Assigning
Let's go over an important part of any data work, accessing our data.

### Naive accessors
Native Python objects provide good way of indexing data, which Pandas carries over to it's objects. Consider the data:

In [ ]:
import pandas as pd
# Import data
import my_modules.data_imports as data
wine_data = data.import_wine_data()

# Peek the data
wine_data

In Python, we can access the property of an object by accessing it as an attribute. A *book* object, for example, might have a *title* property, which we can access by calling ```book.title```. Columns in a pandas DataFrame work in much the same way.

Hence to access the country property of reviews we can use:

In [ ]:
wine_data.country

If we have a Python dictionary, we can access its values using the indexing ```([])``` operator. We can do the same with columns in a DataFrame:

In [ ]:
wine_data['country']

### Indexing in pandas
The indexing operator and attribute selection are nice because they work just like they do in the rest of the Python ecosystem, making them easy to pick up and use. However, pandas has its own accessor operators, ```loc``` and ```iloc```. For more advanced operations, these are the ones we should use.

#### Index-based selection
Pandas indexing works in one of two paradigms. The first is **index-based selection**: selecting data based on its numerical position in the data. iloc follows this paradigm.

To select the first row of data in a DataFrame, we may use the following:

In [ ]:
wine_data.iloc[0]

Both ```loc``` and ```iloc``` are row-first, column-second. This is the opposite of what we do in native Python (and all other languages), which is column-first, row-second.

This means that it's marginally easier to retrieve rows, and marginally harder to get retrieve columns. 
##### iloc
To get a column with iloc, we can do the following:

In [ ]:
wine_data.iloc[:, 0]

On its own, the ```:``` operator, which also comes from native Python, means "everything". When combined with other selectors, however, it can be used to indicate a range of values. For example, to select the *country* column from just the first, second, and third row, we would do:

In [ ]:
wine_data.iloc[:3, 0]

Finally, it's worth knowing that negative numbers can be used in selection. This will start counting forwards from the end of the values. So for example here are the last five elements of the dataset.

In [ ]:
wine_data.iloc[-5:]

#### Label-based selection
The second paradigm for attribute selection is the one followed by the loc operator: **label-based selection**. In this paradigm, it's the data index value, not its position, which matters.

##### loc
For example, to get the first entry in reviews, we would now do the following:

In [ ]:
# get the conutry of row 0
wine_data.loc[0, 'country']

```iloc``` is conceptually simpler than ```loc``` because it ignores the dataset's indices. When we use ```iloc``` we treat the dataset like a big matrix (a list of lists), one that we have to index into by position. ```loc```, by contrast, uses the information in the indices to do its work. Since your dataset usually has meaningful indices, it's usually easier to do things using loc instead. For example, here's one operation that's much easier using loc:

In [ ]:
# get all data in the following columns
wine_data.loc[:, ['taster_name', 'taster_twitter_handle', 'points']]

#### Difference between ```loc``` and ```iloc```
There's one main difference between ```loc``` and ```iloc``` and that's the way they handle their indexing schemas.

```iloc``` uses the Python ```stdlib``` indexing scheme, where the first element of the range is included and the last one excluded. So ```0:10``` will select entries ```0,...,9```. ```loc```, meanwhile, indexes inclusively. So ```0:10``` will select entries ```0,...,10```.

### Manipulating the index
Manipulating the index
Label-based selection derives its power from the labels in the index. Critically, the index we use is not immutable. We can manipulate the index in any way we see fit (or if it wasn't set during ```read_csv```) .

The set_index() method can be used to do the job. Here is what happens when we set_index to the title field:


In [ ]:
wine_data.set_index('title')

### Conditional selection
We can use conditional statements inside of ```loc``` for more interesting ways of selecting data.

In [ ]:
wine_data.loc[wine_data.country == 'Italy']

The above statement pulled ~20,000 rows, while originally there were ~130,000. About 15% of wine comes from Italy!

Now let's find the highly reviewed wines in Italy. Wine is reviewd on an 80-100 point scale, so let's find wines that got atleast a 90.

In [ ]:
wine_data.loc[(wine_data.country == 'Italy') & (wine_data.points >= 90)]

Pandas also comes with a few useful built-in conditional selectors, two of which are:

#### ```isin```
```isin``` lets you select data whose value "is in" a list of values.


In [ ]:
wine_data.loc[wine_data.country.isin(['Italy', 'France'])]

#### ```isnull``` & ```notnull``` 
```isnull``` (and it's friend ```notnull```) let us highlight values which are (or are not) NaN.


In [ ]:
wine_data.loc[wine_data.price.notnull()]

## 3. Summary Functions and Maps
In this section, we'll work on getting out data in the right "shape". Let's go ahead and get some summaries of our data.

### Summary Functions
First, let's consider the ```describe()``` function, which gives us a high-level summary of the attributes of a given column. Note that the ```describe()``` method is type aware and will changes it output based on the input.

In [ ]:
wine_data.points.describe()

In [ ]:
wine_data.taster_name.describe()

Here are some more useful commands:
- ```mean()```: returns the the mean of the specified column
- ```unique()```: returns all unique values of the specified column
- ```value_counts()```: returns all unique values *and* how often they occur in the dataset for the specified column

In [ ]:
wine_data.taster_name.value_counts()

### Maps
Maps are very useful for mapping/transforming out data. Python comes a few different mapping methods, but let's look at the two most useful.

#### ```map()```
This is your good ol' basic map function that will transform values with the given lambda method for the specified column.


In [ ]:
point_mean = wine_data.points.mean()
wine_data.points.map(lambda p: p - point_mean)

#### ```apply()```
This is the DataFrame equivalent of map, that takes the supplied method and applies it on each row.

In [ ]:
def remean_points(row):
    row.points = row.points - point_mean
    return row

wine_data.apply(remean_points, axis='columns')

If ```apply()``` is called with ```axis='index'```, then instead of passing a function to transform each row, we would need to give a function to transform each *column*.

**Note:** Both ```map()``` and ```apply()``` return new, transformed Series/DataFrames, leaving the original intact.

## 4. Grouping and Sorting
Mapping is wonderful for transforming data across Series and DataFrames, but many times we want to group our data. For this, we'll often use the ```groupby()``` method.

### Groupwise analysis
For our first example of ```groupby()``` let's replicate the functionality of ```value_counts()```.

In [ ]:
wine_data.groupby('points').points.count()

The ```groupby()``` method first created a group of reviews which alloted the same points to the given wines. Then, for each group, we grabbed the ```points()``` column and counted how many times it appeared.

We can think of each group we generate as being a slice of our DataFrame containing only data with values that match. This DataFrame is accessible to us directly using the ```apply()``` method, allowing us to then transform/work with these groups.

Let's find the first wine reviewed from each winery.

In [ ]:
wine_data.groupby('winery').apply(lambda df: df.title.iloc[0])

We can also group by more than one column, for example, here's how we would pick out the best wine by country *and* province.

In [ ]:
wine_data.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()])

Another useful grouping method is ```agg()``` which lets us run a bunch of different functions on the DataFrame at once; aggregation!

In [ ]:
wine_data.groupby(['country']).price.agg([len, min, max])

### Multi-indexes
In all of the above examples, we've been working with DataFrame or Series objects with a single-label index. ```groupby()``` is slightly different in the fact that, depending on the operation we run, it will sometimes result in what is called a multi-index.

A multi-index differs from a regular index in that it has multiple levels. For example:

In [ ]:
countries_reviewed = wine_data.groupby(['country', 'province']).description.agg([len])
print(countries_reviewed)
multi_index = countries_reviewed.index
type(multi_index)

Multi-indices have several methods for dealing with their tiered structure which are absent for single-level indices. They also require two levels of labels to retrieve a value. 

The use cases for a multi-index are detailed alongside instructions on using them in the MultiIndex / Advanced Selection section of the pandas documentation.

However, in general the multi-index method you will use most often is the one for converting back to a regular index, the reset_index() method:

In [ ]:
countries_reviewed.reset_index()

### Sorting

Sorting is pretty straight forward. Our previous grouped example ended up sorting by index. We can change the sort using ```sort_values()```. Note that ```sort_values()``` automatically sorts by ascending order. We can also sort the index by using ```sort_index()```.
And finally, we can sort by a column at a time by specifying which columns to use.

In [ ]:
print(countries_reviewed.sort_values(by='len'))
print(colored('==============================================', 'blue'))
print(countries_reviewed.sort_values(by='len', ascending=False))
print(colored('==============================================', 'blue'))
print(countries_reviewed.sort_index())
print(colored('==============================================', 'blue'))
print(countries_reviewed.sort_values(by=['country', 'len']))
print(colored('==============================================', 'blue'))

## 5. Data Types and Missing Values
Let's now talk about typing with DataFrames and how to replace values.

### Dtypes
The data type for a column in a DataFrame or a Series is known as a **dtype**. By using the ```.dtype``` method, we can grab the specific type of a column, whereas we can use ```.dtypes``` to get the type of every column in a DataFrame.

In [ ]:
print(wine_data.price.dtype)
print(colored('====================================', 'blue'))
print(wine_data.dtypes)

One thing to stay aware of is the fact that strings are labeled as *objects* instead of a traditional string.

Another useful function is ```astype()```, which we can use to transform a column of one type to another wherever such a conversion makes sense.

In [ ]:
wine_data.points.astype('float64')

### Missing Data
Entries missing values are given the value *NaN*, short for "Not a Number". For technical reasons these NaN values are always of the float64 dtype.

Pandas provides some methods specific to missing data. To select NaN entries we can use ```pd.isnull()``` (or its companion ```pd.notnull()```). We can use them to select specific rows from our dataset.

In [ ]:
wine_data[pd.isnull(wine_data.country)]

Replacing missing values is a common operation. Pandas provides a really handy method for this problem: ```fillna()```. ```fillna()``` provides a few different strategies for mitigating such data. For example, we can simply replace each NaN with an "Unknown":

In [ ]:
wine_data.region_2.fillna("Unknown")

In [ ]:
wine_data.taster_twitter_handle.replace("@kerinokeefe", "@kerino")

Another handy method used for replacing certain values, is of course,```replace()```. Say one of the reviewers changed their twitter handles.

## 6. Renaming and Combining
In this final section, let's go over working with column names, index names, and other names.

### Renaming
The first function to look at is ```rename()```, which lets us change index and/or column names. Let's take the *points* column and rename it to *score*:


In [ ]:
wine_data.rename(columns={'points':'score'})

The ```rename()``` function let's us rename the index *or* column by specifiying an *index* or *column* keyword parameter. The function supports a variety of input formats, but usually a Python dictionary is the most convenient. Let's rename the index for example.

In [ ]:
wine_data.rename(index={0: 'firstEntry', 1: 'secondEntry'})

We'll probably be renaming columns very often, but renaming index values is rather rare. For that, we'll be using ```set_index()``` instead.

Both the row index and the column index can have their own name attribute. The complimentary ```rename_axis()``` method may be used to change these names. For example:

In [ ]:
wine_data.rename_axis("wines", axis='rows').rename_axis("fields", axis="columns")

### Combining
When performing operations on a dataset, we will sometimes need to combine different DataFrames and/or Series in non-trivial ways. Pandas has three core methods for doing this. In order of increasing complexity, these are ```concat()```, ```join()```, and ```merge()```. Most of what ```merge()``` can do can also be done more simply with ```join()```, so we will omit it and focus on the first two functions here.

The simplest combining method is concat(). Given a list of elements, this function will smush those elements together along an axis.

This is useful when we have data in different DataFrame or Series objects but having the same fields (columns). One example: the YouTube Videos dataset, which splits the data up based on country of origin (e.g. Canada and the UK, in this example). If we want to study multiple countries simultaneously, we can use ```concat()``` to smush them together:

In [ ]:
canada_youtube = data.import_youtube_ca_data()
print(colored("Canada Youtube:", 'green'))
print(canada_youtube.head())

print(colored("British Youtube:", 'green'))
british_youtube = data.import_youtube_gb_data()
print(british_youtube.head())

print(colored("Concat data:", 'blue'))
pd.concat([canada_youtube, british_youtube])

The middlemost combiner in terms of complexity is ```join()```. ```join()``` lets you combine different DataFrame objects which have an index in common. For example, to pull down videos that happened to be trending on the same day in both Canada and the UK, we could do the following:

In [ ]:
left = canada_youtube.set_index(['title', 'trending_date'])
right = canada_youtube.set_index(['title', 'trending_date'])

left.join(right, lsuffix='_CAN', rsuffix='_UK')

The *lsuffix* and *rsuffix* parameters are necessary here because the data has the same column names in both British and Canadian datasets. If this wasn't true (because, say, we'd renamed them beforehand) we wouldn't need them.